# Observed Taylor law: variance vs mean 
**in protein coding gene length distributions for the different species**

## Import python modules

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
# 
#
system = list(os.uname())[0]
if system == 'Linux':
    base_dir = "/media/emuro/Nubya/"
elif system == 'Darwin':
    base_dir = "/Volumes/Nubya/"
#
# init some data paths
working_on_extra_path  = "../../../"
main_tables_path       = working_on_extra_path + "main_tables/"
working_on_tables_path = working_on_extra_path + "working_on_tables/"
# 
# ensembl (to obtain the genome accession)
ensembl_Id_path = base_dir + "results/geneLength/outputInputFiles/" + "some_tables/species_Ensembl_taxid/" 
#
# NCBI genome quality
home = str(Path.home())  #; print(home)
NCBI_GENOME_PATH = home + "/Desktop/goingOn/geneLength/NCBI_genomeReports/"

## Retrieving the statistical descriptions of the gene length's distributions for the different genomes

In [2]:
# statistics on length distribution for different species
stat_file = main_tables_path + "stat_protCodGenes.tsv"
print("The statistical descriptions of the protein coding gene distributions for the different species is in:\n", stat_file, "\n")

# retrieve data and diminish the number of columns
stat_df = pd.read_csv(stat_file, low_memory=False, sep="\t")
# In case you want a simple table:
# stat_df = stat_df[["division_7", "division_8", "species", "assembly", "taxonomy_id", "Lineage"]] 

# visualize the data frame
if 1:
    print(stat_df.shape)
    print(stat_df.value_counts('division_7', dropna=False))
    pd.set_option('display.max_columns', stat_df.shape[1])
    display(stat_df.head(2))

The statistical descriptions of the protein coding gene distributions for the different species is in:
 ../../../main_tables/stat_protCodGenes.tsv 

(33627, 41)
division_7
bacteria       31943
fungi           1014
protists         237
vertebrates      222
metazoa          115
plants            96
Name: count, dtype: int64


,index_original,division_7,species,assembly,trunk_genes_path,genes_file,count,mean,std,var,min,25perc,50perc,75perc,max,log10_mean,log10_std,log10_var,log10_min,log10_25perc,log10_50perc,log10_75perc,log10_max,log_mean,log_std,log_var,log_min,log_25perc,log_50perc,log_75perc,log_max,taxonomy_id,genebuild,Scientific_name,Rank,Lineage,division_4_byLineage,Virus_host,division_8,bool_Rank,sameTax_order
0,7068,bacteria,methanobacterium_bryantii_gca_002287175,ASM228717v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_bryantii...,3168,840.404040,649.879873,422343.848699,131,416.0,686.0,1079.0,7910,2.827684,0.287565,0.082694,2.117271,2.619093,2.836324,3.033021,3.898176,6.510982,0.662143,0.438433,4.875197,6.030685,6.530878,6.98379,8.975883,2161,2017.0,Methanobacterium bryantii,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1
1,26156,bacteria,methanobacterium_formicicum_gca_000762265,ASM76226v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_formicic...,2352,862.427296,589.270407,347239.612747,107,458.0,728.0,1115.0,6779,2.849723,0.276255,0.076317,2.029384,2.660865,2.862131,3.047275,3.831166,6.561730,0.636100,0.404623,4.672829,6.126869,6.590301,7.01661,8.821585,2162,2014.0,Methanobacterium formicicum,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1


## Python functions

### All protein coding genes from the paper (33,627)
incluyendo 168 entradas de division_7 que se pierden al hacer el label division_8.

## Filtro por calidad de Genomas: usando NCBI genome annotations

#### Get good genomes
Genome status: Complete Genome or Chromosomes (from NCBI genome annotation)

In [3]:
# Prokarya genomes: get those with a good genome annotation ("Status")
#
# Get well annotated eukarya genomes
g_prok_df = pd.read_csv(NCBI_GENOME_PATH + "prokaryotes.txt", low_memory=False, sep="\t") # tax_id, status, accession 
g_euk_df  = pd.read_csv(NCBI_GENOME_PATH + "eukaryotes.txt", sep="\t") 
print("From NCBI, before filtering genomes:\n\tprokaryotes", g_prok_df.shape, "\n\teukaryotes", g_euk_df.shape)
# Relevant columns: col2 = "TaxID"; col19 (prokarya) col9 (eukarya) = "Assembly Accession"
#                   col16 (prokarya) col17 (eukarya) = "Status"
good_status = ["Complete Genome", "Chromosome"] #It can be: ["Complete Genome", "Chromosome", "Scaffold", "Contig"]

# Genome quality filter!
if 0: # decide to filter or not
    g_prok_df = g_prok_df[ g_prok_df["Status"].isin(good_status)]
    g_euk_df  = g_euk_df[  g_euk_df["Status"].isin(good_status)]
    print("From NCBI, after filtering genomes:\n\tprokaryotes", g_prok_df.shape, "\n\teukaryotes", g_euk_df.shape)

From NCBI, before filtering genomes:
	prokaryotes (405092, 23) 
	eukaryotes (22897, 19)


### From NCBI 
Assemblies accessions as keys: 
- dicts: Groups (taxon), Subgroups of organisms (NCBI)
- dicts: TaxID, GC\%, size (Mbp), replicons, status

In [4]:
if 0: # visualize columns 
    print("Columns of NCBI genome files:")
    print("Eukarya:", g_euk_df.columns, "Prokarya:", g_prok_df.columns, sep="\n", end="\n\n")
    
print("With all the NCBI genomes that I need: creating dictionaries from the Assembly accession key for each genome:")
#
# Eukaryotes
dict_euk_groupOfAssembly     = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["Group"]))         
dict_euk_subgroupOfAssembly  = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["SubGroup"]))    
dict_euk_taxidOfAssembly     = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["TaxID"]))          # print(dict_euk_taxidOfAssembly)
dict_euk_gcOfAssembly        = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["GC%"]))            
dict_euk_sizeOfAssembly      = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["Size (Mb)"]))      
dict_euk_repliconsOfAssembly = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["Replicons"]))     
dict_euk_statusOfAssembly    = dict(zip(g_euk_df["Assembly Accession"], g_euk_df["Status"]))         
if 1:
  print("\tEukaryotes:", len(dict_euk_statusOfAssembly))

# Prokaryotes
dict_groupOfAssembly     = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["Group"]))         
dict_subgroupOfAssembly  = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["SubGroup"]))    
dict_taxidOfAssembly     = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["TaxID"]))            # print(dict_taxidOfAssembly)
dict_gcOfAssembly        = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["GC%"]))            
dict_sizeOfAssembly      = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["Size (Mb)"]))       
dict_repliconsOfAssembly = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["Replicons"]))     
dict_statusOfAssembly    = dict(zip(g_prok_df["Assembly Accession"], g_prok_df["Status"]))        
if 1:
  print("\tProkaryotes:", len(dict_statusOfAssembly))
    
# Combine both dictionaries for Eukaryotes and Prokaryotes
dict_groupOfAssembly.update(    dict_euk_groupOfAssembly)
dict_subgroupOfAssembly.update( dict_euk_subgroupOfAssembly)
dict_taxidOfAssembly.update(    dict_euk_taxidOfAssembly)
dict_gcOfAssembly.update(       dict_euk_gcOfAssembly)
dict_sizeOfAssembly.update(     dict_euk_sizeOfAssembly)
dict_repliconsOfAssembly.update(dict_euk_repliconsOfAssembly)
dict_statusOfAssembly.update(   dict_euk_statusOfAssembly)
if 1:
  print("\tCombined, Eukaryotes & Prokaryotes:", len(dict_statusOfAssembly))

With all the NCBI genomes that I need: creating dictionaries from the Assembly accession key for each genome:
	Eukaryotes: 22897
	Prokaryotes: 405092
	Combined, Eukaryotes & Prokaryotes: 427989


### From Ensembl Tax_id file 
dict: Assembly_accession of species (species is a perfect id in this case)

In [5]:
# ENSEMBL assembly accession
# system
import os
system = list(os.uname())[0]
if system == 'Linux':
    base_dir = "/media/emuro/Nubya/"
elif system == 'Darwin':
    base_dir = "/Volumes/Nubya/"
ensembl_Id_file = base_dir + "results/geneLength/outputInputFiles/" + "some_tables/species_Ensembl_taxid/" + "species_Ensembl_EMv2.0.tsv" # with covid
print(ensembl_Id_file)

# retrieve data
id_df = pd.read_csv(ensembl_Id_file, sep="\t")
id_df = id_df[["species", "assembly_accession"]]

dict_assemblyOfSpecies = dict(zip(id_df["species"], id_df["assembly_accession"])) # print(len(dict_assemblyOfSpecies))
# visualize data
if 0: #dict
    print(dict_assemblyOfSpecies)
if 0: #df
    pd.set_option('display.max_columns', stat_df.shape[1])
    display(id_df.head(2))
    print(id_df.shape)

# Finally:
# Add the assembly of the species to the main df
################################################
stat_df["ensembl_assembly_accession"] = stat_df["species"].map(dict_assemblyOfSpecies)
if 1:
    display(stat_df.head(2))
    print(stat_df.shape)       

/media/emuro/Nubya/results/geneLength/outputInputFiles/some_tables/species_Ensembl_taxid/species_Ensembl_EMv2.0.tsv


,index_original,division_7,species,assembly,trunk_genes_path,genes_file,count,mean,std,var,min,25perc,50perc,75perc,max,log10_mean,log10_std,log10_var,log10_min,log10_25perc,...,log10_max,log_mean,log_std,log_var,log_min,log_25perc,log_50perc,log_75perc,log_max,taxonomy_id,genebuild,Scientific_name,Rank,Lineage,division_4_byLineage,Virus_host,division_8,bool_Rank,sameTax_order,ensembl_assembly_accession
0,7068,bacteria,methanobacterium_bryantii_gca_002287175,ASM228717v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_bryantii...,3168,840.404040,649.879873,422343.848699,131,416.0,686.0,1079.0,7910,2.827684,0.287565,0.082694,2.117271,2.619093,...,3.898176,6.510982,0.662143,0.438433,4.875197,6.030685,6.530878,6.98379,8.975883,2161,2017.0,Methanobacterium bryantii,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1,GCA_002287175.1
1,26156,bacteria,methanobacterium_formicicum_gca_000762265,ASM76226v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_formicic...,2352,862.427296,589.270407,347239.612747,107,458.0,728.0,1115.0,6779,2.849723,0.276255,0.076317,2.029384,2.660865,...,3.831166,6.561730,0.636100,0.404623,4.672829,6.126869,6.590301,7.01661,8.821585,2162,2014.0,Methanobacterium formicicum,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1,GCA_000762265.1


(33627, 42)


#### Taxid and assembly accesssion with high quality genome

In [6]:
# taxid with high quality genome assemblies
taxid_highqu_genome_l    = list(dict_taxidOfAssembly.values()) # taxids
assembly_highqu_genome_l = list(dict_taxidOfAssembly.keys())   # assemblies

reduced_stat_df = stat_df.copy()
if 0: # to reduce by taxonomy_id is not necessary now
    reduced_stat_df = stat_df[stat_df["taxonomy_id"].isin(taxid_highqu_genome_l)].copy() # reduce to hquality taxonomies...
    print("By taxonomy,", reduced_stat_df.shape) # much more redundancy, the assembly commands
#
reduced_stat_df = reduced_stat_df[reduced_stat_df["ensembl_assembly_accession"].isin( assembly_highqu_genome_l )].copy()
print("Also by assembly accession,", reduced_stat_df.shape)

Also by assembly accession, (32022, 42)


#### For each filtered species add: genome size, gc content, chromosomes,... 

In [7]:
# Add the assembly of the species


# New columns initialization
stat_df["ensembl_assembly_accession"] = np.nan
stat_df["group"]           = np.nan
stat_df["subgroup"]        = np.nan
stat_df["gc_percent"]      = np.nan
stat_df["size_Mbp"]        = np.nan
stat_df["assembly_status"] = np.nan
stat_df["chromosomes"]     = np.nan

stat_df["ensembl_assembly_accession"] = stat_df["species"].map(dict_assemblyOfSpecies) # column to link dbs
# Add more genomic info
stat_df["group"]      = stat_df["ensembl_assembly_accession"].map(dict_groupOfAssembly)
stat_df["subgroup"]        = stat_df["ensembl_assembly_accession"].map(dict_subgroupOfAssembly)
stat_df["gc_percent"]      = stat_df["ensembl_assembly_accession"].map(dict_gcOfAssembly)
stat_df["size_Mbp"]        = stat_df["ensembl_assembly_accession"].map(dict_sizeOfAssembly)
stat_df["assembly_status"] = stat_df["ensembl_assembly_accession"].map(dict_statusOfAssembly)
stat_df["chromosomes"]     = stat_df["ensembl_assembly_accession"].map(dict_repliconsOfAssembly)

#stat_df = stat_df[stat_df.division_8 != "bacteria"]
#stat_df = stat_df[stat_df.division_8 != "archaea"]
#stat_df = stat_df[stat_df.division_8 != "fungi"]
#stat_df = stat_df[stat_df.division_8 != "plants"]
#stat_df = stat_df[stat_df.division_8 != "protists"]
#stat_df = stat_df[stat_df.division_8 != "archaea"]
##stat_df = stat_df[stat_df.subgroup == "Birds"]
#stat_df = stat_df[stat_df.subgroup == "Fishes"]

display(stat_df.head(2))
print(stat_df.shape)
print(stat_df['ensembl_assembly_accession'].value_counts(dropna=False))

,index_original,division_7,species,assembly,trunk_genes_path,genes_file,count,mean,std,var,min,25perc,50perc,75perc,max,log10_mean,log10_std,log10_var,log10_min,log10_25perc,...,log_50perc,log_75perc,log_max,taxonomy_id,genebuild,Scientific_name,Rank,Lineage,division_4_byLineage,Virus_host,division_8,bool_Rank,sameTax_order,ensembl_assembly_accession,group,subgroup,gc_percent,size_Mbp,assembly_status,chromosomes
0,7068,bacteria,methanobacterium_bryantii_gca_002287175,ASM228717v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_bryantii...,3168,840.404040,649.879873,422343.848699,131,416.0,686.0,1079.0,7910,2.827684,0.287565,0.082694,2.117271,2.619093,...,6.530878,6.98379,8.975883,2161,2017.0,Methanobacterium bryantii,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1,GCA_002287175.1,Euryarchaeota,Methanomada group,33.2,3.46637,Contig,-
1,26156,bacteria,methanobacterium_formicicum_gca_000762265,ASM76226v1,ftp.ensemblgenomes.org/pub/bacteria/release-49...,protein_coding.genes.methanobacterium_formicic...,2352,862.427296,589.270407,347239.612747,107,458.0,728.0,1115.0,6779,2.849723,0.276255,0.076317,2.029384,2.660865,...,6.590301,7.01661,8.821585,2162,2014.0,Methanobacterium formicicum,Species,Archaea; Euryarchaeota; Methanomada group; Met...,Archaea,NaN,archaea,1,1,GCA_000762265.1,Euryarchaeota,Methanomada group,41.3,2.44999,Complete Genome,chromosome:NZ_CP006933.1/CP006933.1


(33627, 48)
ensembl_assembly_accession
NaN                22
GCA_901326685.1     2
GCA_003481565.1     2
GCA_004771295.1     2
GCA_003480145.1     2
                   ..
GCA_005518865.1     1
GCA_004798205.1     1
GCA_001955065.1     1
GCA_003206495.1     1
GCA_005862185.1     1
Name: count, Length: 32924, dtype: int64


In [8]:
# Save the file if desired
if 0:
    stat_df.to_csv(working_on_tables_path + "stat_protCodGenes_with_ncbiGenomeData.tsv", sep="\t", index=False, header=True)